In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

In [2]:
import helper
%aimport helper

helper = helper.Helper()

In [3]:
import tensorflow as tf
import numpy as np
import os

## Load the data

In [4]:
data_dir = np.load('./data/train_test_data.npz')
X_train = data_dir['X_train']
X_test = data_dir['X_test']
y_train = data_dir['y_train']
y_test = data_dir['y_test']
data = np.concatenate((X_train, X_test))
labels = np.concatenate((y_train, y_test))

## Load models and histories

In [5]:
model_name0 = "Head only"
model0 = helper.loadModel(model_name0)
score0 = model0.evaluate(X_test, y_test, verbose=0)
history0 = helper.loadHistory(model_name0)

model_name1 = "Dense + Head"
model1 = helper.loadModel(model_name1)
score1 = model1.evaluate(X_test, y_test, verbose=0)
history1 = helper.loadHistory(model_name1)

### 1. Model structure test

In [6]:
### BEGIN HIDDEN TESTS
num_cases = np.unique(labels).shape[0]

if labels.ndim == 2 and num_cases >=2:
    activation = "softmax"
else:
    activation = "sigmoid"
    num_cases = 1

## Model0 Head Only
assert len(model0.layers) <= 3

n_samples, width, height, channel = data.shape
dense_layer = model0.get_layer(name='dense_head')
# Dense layer tests
assert dense_layer.input.shape[1:] == tf.TensorShape([width * height * channel])
assert dense_layer.output.shape[1:] == tf.TensorShape([num_cases])
### END HIDDEN TESTS

In [7]:
### BEGIN HIDDEN TESTS
num_features_1 = 32

dense_1 = model1.get_layer(name='dense_1')
dense_head = model1.get_layer(name='dense_head')

# Dense 1
assert dense_1.input.shape[1:] == tf.TensorShape([width * height * channel])
assert dense_1.output.shape[1:] == tf.TensorShape([num_features_1])

# Dense head
assert dense_head.output.shape[1:] == tf.TensorShape([num_cases])

### END HIDDEN TESTS

### 2. Training result test

In [10]:
### BEGIN HIDDEN TESTS
# get the keyname of accuracy
acc_string = "accuracy" if "accuracy" in history0.keys() else "acc"

assert history0[acc_string][-1] > 0.6
assert history0['val_'+acc_string][-1] > 0.6
### END HIDDEN TESTS

In [11]:
### BEGIN HIDDEN TESTS
assert history1[acc_string][-1] > 0.75
assert history1['val_'+acc_string][-1] > 0.75
### END HIDDEN TESTS

In [12]:
### BEGIN HIDDEN TESTS
assert history1[acc_string][-1] > history0[acc_string][-1]
assert history1['val_'+acc_string][-1] > history0['val_'+acc_string][-1]
### END HIDDEN TESTS

AssertionError: 

### 3. Model parameters number test

In [ ]:
### BEGIN HIDDEN TESTS
assert model0.count_params() == (width * height * channel * num_cases + num_cases)
### END HIDDEN TESTS

In [ ]:
### BEGIN HIDDEN TESTS
dense_layers = [layer for layer in model1.layers if 'dense' in layer.name]
num_new_parameters = (width * height * channel + 1) * num_features_1 + (num_features_1 + 1) * num_cases

if len(dense_layers) == 2:
    assert model1.count_params() == num_new_parameters
elif len(dense_layers) > 2:
    assert model1.count_params() > num_new_parameters
### END HIDDEN TESTS

### 4. Evaluate the model

In [ ]:
### BEGIN HIDDEN TESTS
assert score0[1] > 0.6
### END HIDDEN TESTS

In [ ]:
### BEGIN HIDDEN TESTS
assert score1[1] > 0.8
### END HIDDEN TESTS

In [ ]:
### BEGIN HIDDEN TESTS
assert score1[1] >= score0[1]
### END HIDDEN TESTS